In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from pclib.nn.models import SmallLinearClassifier, PCLeNetV1, PCLeNetV2
from pclib.optim.train import train
from customdataset import PreloadedDataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
VAL_RATIO = 0.1
# transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),                                
# ])
transform = transforms.ToTensor()

dataset = datasets.MNIST('../Datasets/', train=True, download=False, transform=transforms.ToTensor())
# shorten dataset
# length = 100
# dataset = torch.utils.data.Subset(dataset, range(length))
val_len = int(len(dataset) * VAL_RATIO)
train_len = len(dataset) - val_len
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, val_len])
train_dataset = PreloadedDataset.from_dataset(train_dataset, transform, device)
val_dataset = PreloadedDataset.from_dataset(val_dataset, transforms.ToTensor(), device)
INPUT_SHAPE = 784
NUM_CLASSES = 10

In [18]:
from pclib.nn.layers import PCLinearUniweighted as PCLinear

class pclinear(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(pclinear, self).__init__()
        layers = []
        layers.append(PCLinear(input_size, output_size, relu_errs=False, bias=False))

        self.layers = torch.nn.ModuleList(layers)
        self.steps = 5
        self.device = device
        self.num_classes=10
    
    def step(self, x, R, E, y=None):
        if y is not None:
            assert y.shape == R[-1].shape
            R[-1] = y
        R[0], E[0] = self.layers[0](x, R[0], None)
        return R, E
    
    def init_vars(self, batch_size: int):
        R = []
        E = []
        r, e = self.layers[0].init_vars(batch_size, bias=True)
        R.append(r)
        E.append(e)
        return R, E

    def to(self, device):
        self.device = device
        for layer in self.layers:
            layer.to(device)
        return self
    
    def forward(self, x, y=None, steps=None, full_data=False):
        assert len(x.shape) == 2, f"Input must be 2D, got {len(x.shape)}D"
        if steps is None:
            steps = self.steps

        R, E = self.init_vars(x.shape[0])

        # Setting it here and after each step so out-target error is 0 if y is not none
        if y is not None:
            assert y.shape == R[-1].shape
            R[-1] = y

        for _ in range(steps):
            R, E = self.step(x, R, E)
            if y is not None:
                R[-1] = y

        out = R[-1]

        if full_data:
            return out, R, E

        return out

In [19]:
seed = 42
torch.manual_seed(seed)

model_name = "mnist_pclinear"
LEARNING_RATE = 0.001
model = pclinear(INPUT_SHAPE, NUM_CLASSES).to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()
step = 0

In [20]:
# Train Loop

NUM_EPOCHS = 20
step = train(
    model, 
    train_dataset, 
    val_dataset, 
    optimiser, 
    criterion,
    model_name, 
    NUM_EPOCHS, 
    LEARNING_RATE, 
    log_dir="mnist/out/logs",
    model_dir="mnist/out/weights",
    flatten=True, 
    step=step, 
    device=device,
    batch_size=128,
)

KeyboardInterrupt: 

In [13]:

# print mean and std of weights
for layer in model.layers:
    print(layer)
    print(f"layer.weight.mean(): {layer.weight.mean()}")
    print(f"layer.weight.std(): {layer.weight.std()}")
    print(f"layer.bias.mean(): {layer.bias.mean()}")
    print(f"layer.bias.std(): {layer.bias.std()}")

PCLinearUniweighted()
layer.weight.mean(): 0.3066323697566986
layer.weight.std(): 0.23584160208702087
layer.bias.mean(): 0.4673560559749603
layer.bias.std(): 0.2265135794878006


In [9]:
# print mean and std of weights
for layer in model.layers:
    print(layer)
    print(f"layer.weight.mean(): {layer.weight.mean()}")
    print(f"layer.weight.std(): {layer.weight.std()}")
    print(f"layer.bias.mean(): {layer.bias.mean()}")
    print(f"layer.bias.std(): {layer.bias.std()}")

PCLinearUniweighted()
layer.weight.mean(): 0.03455418720841408
layer.weight.std(): 0.14680886268615723
layer.bias.mean(): 0.09652074426412582
layer.bias.std(): 0.13295072317123413
